# OperaManager Example Notebook

In [13]:
# --- Imports ---
import pymif.microscope_manager as mm
import napari
import os
import time

In [14]:
# --- Set input data path ---
main_path = "/g/mif/people/gritti/code/pymif_test_data"
dataset_name = "opera/2C-430Z.ome.tiff"
viventis_path = f"{main_path}/{dataset_name}"  # Update this to your real path


In [15]:
# --- Initialize manager ---
dataset = mm.OperaManager(path=viventis_path, chunks=(1,1,64,1024,1024))

In [16]:
# --- Show metadata summary ---
for i in dataset.metadata:
    print(f"{i.upper()}: {dataset.metadata[i]}")
print("CHUNK SIZE:", dataset.chunks)

SIZE: [(430, 2160, 4181), (430, 1080, 2090), (430, 540, 1045)]
SCALES: [(0.7, 0.285388764949, 0.285388764949), (0.7, 0.570777529898, 0.570914079546301), (0.7, 1.141555059796, 1.141828159092602)]
UNITS: ['µm', 'µm', 'µm']
TIME_INCREMENT: 1.0
TIME_INCREMENT_UNIT: s
CHANNEL_NAMES: ['Fluorescein (FITC)', 'Brightfield_copy']
CHANNEL_COLORS: [16711935, 8421631]
DTYPE: uint16
AXES: tczyx
CHUNK SIZE: (1, 1, 64, 1024, 1024)


In [17]:
dataset.data[0]

dask.array<rechunk-merge, shape=(1, 2, 430, 2160, 4181), dtype=uint16, chunksize=(1, 1, 64, 1024, 1024), chunktype=numpy.ndarray>

In [18]:
# --- Visualize in Napari ---
# Opens napari with pyramid-aware viewing and color channels
# NOT RECOMMENDED FOR VERY LARGE DATASETS!
# To visualize lower resolution, build the pyramid and use the start_level argument (last cell).
# To visualize the whole dataset at full resolution, convert data into ome-zarr

# viewer = dataset.visualize()

In [7]:
# --- Visualize in Napari ---
# Opens napari with pyramid-aware viewing and color channels, generate lower resolution and load it into memory
viewer = dataset.visualize( start_level = 2, in_memory = True )
# NOTE: This fails because the dataset only has 1 resolution layer

In [9]:
# --- Build pyramid if not already ---
dataset.build_pyramid(num_levels=7, downscale_factor=2)

# --- Optional: Write to OME-Zarr format ---
output_zarr_path = f"{main_path}/opera_2C-430Z.zarr"
dataset.write(output_zarr_path)

Requested start level 0
Resolution layer already available


In [10]:
# --- Show metadata summary ---
for i in dataset.metadata:
    print(f"{i.upper()}: {dataset.metadata[i]}")
print("CHUNK SIZE:", dataset.chunks)

SIZE: [(1, 2, 430, 2160, 4181), (1, 2, 215, 1080, 2091), (1, 2, 108, 540, 1046), (1, 2, 54, 270, 523), (1, 2, 27, 135, 262), (1, 2, 14, 68, 131), (1, 2, 7, 34, 66)]
SCALES: [(0.7, 0.285388764949, 0.285388764949), (1.4, 0.570777529898, 0.570777529898), (2.8, 1.141555059796, 1.141555059796), (5.6, 2.283110119592, 2.283110119592), (11.2, 4.566220239184, 4.566220239184), (22.4, 9.132440478368, 9.132440478368), (44.8, 18.264880956736, 18.264880956736)]
UNITS: ['µm', 'µm', 'µm']
TIME_INCREMENT: 1.0
TIME_INCREMENT_UNIT: s
CHANNEL_NAMES: ['Fluorescein (FITC)', 'Brightfield_copy']
CHANNEL_COLORS: [16711935, 8421631]
DTYPE: uint16
AXES: tczyx
CHUNK SIZE: (1, 1, 64, 1024, 1024)


# Now we use the ZarrManager to load the new data and display it

In [19]:
input_zarr_path = f"{main_path}/opera_2C-430Z.zarr"

In [20]:
# --- Initialize manager ---
# Here, the argument chunks can be used to specify a preferred chunk (Default: (1,1,16,256,256)
# Here, we use the same chunks.
zarr_dataset = mm.ZarrManager(path=input_zarr_path, chunks=(1,1,64,1024,1024))

In [22]:
# --- Show metadata summary ---
for i in dataset.metadata:
    print(f"{i.upper()}: {zarr_dataset.metadata[i]}")
print("CHUNK SIZE:", zarr_dataset.chunks)

SIZE: [(1, 2, 430, 2160, 4181), (1, 2, 215, 1080, 2091), (1, 2, 108, 540, 1046), (1, 2, 54, 270, 523), (1, 2, 27, 135, 262), (1, 2, 14, 68, 131), (1, 2, 7, 34, 66)]
SCALES: [(0.7, 0.285388764949, 0.285388764949), (1.4, 0.570777529898, 0.570777529898), (2.8, 1.141555059796, 1.141555059796), (5.6, 2.283110119592, 2.283110119592), (11.2, 4.566220239184, 4.566220239184), (22.4, 9.132440478368, 9.132440478368), (44.8, 18.264880956736, 18.264880956736)]
UNITS: ('micrometer', 'micrometer', 'micrometer')
TIME_INCREMENT: 1.0
TIME_INCREMENT_UNIT: s
CHANNEL_NAMES: ['Fluorescein (FITC)', 'Brightfield_copy']
CHANNEL_COLORS: [16711935, 8421631]
DTYPE: uint16
AXES: tczyx
CHUNK SIZE: (1, 1, 64, 1024, 1024)


In [23]:
zarr_dataset.data[0]

dask.array<from-zarr, shape=(1, 2, 430, 2160, 4181), dtype=uint16, chunksize=(1, 1, 64, 1024, 1024), chunktype=numpy.ndarray>

In [24]:
# --- Visualize in Napari ---
# Opens napari with pyramid-aware viewing and color channels
# It reads only the visualizing slice. Faster but not interactive.
start = time.time()
viewer = zarr_dataset.visualize()
print(f"{time.time()-start}")

1.9682822227478027


In [19]:
start = time.time()
viewer = zarr_dataset.visualize(                        
                        start_level = 0, 
                        stop_level = 2, 
                        in_memory = True 
)
print(f"{time.time()-start}")

12.473992109298706
